In [1]:
query = """
{
  propositions(first: 100000) {
    totalCount
    edges {
      node {
        # title
        # status
        governance: governance {
          platform_deployment_id: id
          # platform_deployment_name: name
        }
        phases {
          __typename
          ...on GovernorProposal {
          	proposal_id: id
            # state
            start {
              date_as_epoch: timestamp
            }
            end {
              date_as_epoch: timestamp
            }
            proposer {
              author: id
            }
            voteTally {
              # __typename
              votes_count: ballotCount
            }
            description
        	}
        }
        # proposal_id: id
      }
    }
  }
}

# x platform: governor
# x platform_deployment_id: governance id
# x proposal_id
# x author: from GovernorProposal
# x date -- this should be as a pandas date, not as an epoch
# x votes_count: ballotCount from GovernorProposal phase
"""

In [2]:
# hit gql https://graphql.messari.io/query
import requests

response = requests.post('https://graphql.messari.io/query', json={'query': query})
print(response.status_code)
data = response.json()
len(data['data']['propositions']['edges'])

200


10773

In [3]:
import pandas as pd
# df from edges
df = pd.DataFrame(data['data']['propositions']['edges'])
# expand the node column
df = pd.concat([df.drop(['node'], axis=1), df['node'].apply(pd.Series)], axis=1)
df.head()

,governance,phases
0,{'platform_deployment_id': 'e229944a-420a-4945...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
1,{'platform_deployment_id': '612f3113-2625-4175...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
3,{'platform_deployment_id': 'dc6a6bc7-baa3-4b12...,[{'__typename': 'GovernorLiteProposal'}]
4,{'platform_deployment_id': '6d02d511-788b-41c5...,[{'__typename': 'SnapshotProposal'}]


In [4]:
# explode phases column
phases_df = df.explode('phases')
phases_df.head()

,governance,phases
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'}
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'}
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'}


In [5]:
# rename phases to phase
phases_df.rename(columns={'phases': 'phase'}, inplace=True)
phases_df.phase.value_counts()

{'__typename': 'PreliminaryDiscussion'}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [6]:
phases_df.head()

,governance,phase
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'}
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'}
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'}


In [7]:
# drop where phase is nan
phases_df.dropna(subset=['phase'], inplace=True)

In [8]:
# pull out typename into new col
phases_df['typename'] = phases_df['phase'].apply(lambda x: x.get('__typename'))
phases_df.head()

,governance,phase,typename
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'},SnapshotProposal
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'},SnapshotProposal
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion


In [9]:
# keep only where typename is 'GovernorProposal'
governor_proposals = phases_df[phases_df['typename'] == 'GovernorProposal']
governor_proposals.head()

,governance,phase,typename
26,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
40,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
58,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
82,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
114,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal


In [10]:
governor_proposals.shape

(985, 3)

In [11]:
# x platform: governor
# x platform_deployment_id: governing_body_id
# x proposal_id
# x author: from GovernorProposal
# x date -- this should be as a pandas date, not as an epoch
# x start -- this should be as a pandas date, not as an epoch
# x end -- this should be as a pandas date, not as an epoch
# x votes_count: ballotCount from GovernorProposal phase

In [12]:
# expand phase dict column into separate columns
governor_proposals = pd.concat([governor_proposals.drop(['phase'], axis=1), governor_proposals['phase'].apply(pd.Series)], axis=1)
governor_proposals.head()

,governance,typename,__typename,proposal_id,start,end,proposer,voteTally,description
26,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,GovernorProposal,GovernorProposal,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'date_as_epoch': 1666434539},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124},\n\n# Simple Summary\n\nThis proposal updates ...
40,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,GovernorProposal,GovernorProposal,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'date_as_epoch': 1678790255},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98},# [S17] Gitcoin Allo Budget Proposal\nThis a r...
58,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,GovernorProposal,GovernorProposal,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'date_as_epoch': 1663550339},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9},IIP-26: Add senior tranche of cpWIN-USDC PYT a...
82,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,GovernorProposal,GovernorProposal,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'date_as_epoch': 1674702539},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413},\n\n# Simple Summary\n\nThis governance propos...
114,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,GovernorProposal,GovernorProposal,0xec568fffba86c094cf06b22134b23074dfe2252c-348,{'date_as_epoch': 1697645219},{'date_as_epoch': 1697877215},{'author': '0x329c54289ff5d6b7b7dae13592c6b1ed...,{'votes_count': 191},"---\ntitle: ""Enable borrow of OP token""\nautho..."


In [13]:
# drop typename and __typename cols
governor_proposals = governor_proposals.drop(columns=['typename', '__typename'])
governor_proposals.head()

,governance,proposal_id,start,end,proposer,voteTally,description
26,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'date_as_epoch': 1666434539},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124},\n\n# Simple Summary\n\nThis proposal updates ...
40,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'date_as_epoch': 1678790255},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98},# [S17] Gitcoin Allo Budget Proposal\nThis a r...
58,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'date_as_epoch': 1663550339},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9},IIP-26: Add senior tranche of cpWIN-USDC PYT a...
82,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'date_as_epoch': 1674702539},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413},\n\n# Simple Summary\n\nThis governance propos...
114,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-348,{'date_as_epoch': 1697645219},{'date_as_epoch': 1697877215},{'author': '0x329c54289ff5d6b7b7dae13592c6b1ed...,{'votes_count': 191},"---\ntitle: ""Enable borrow of OP token""\nautho..."


In [14]:
# * platform
# * platform_deployment_id
# * proposal_id
# * author
# * date -- this should be as a pandas date, not as an epoch
# * votes_count

In [15]:
governor_proposals['platform_deployment_id'] = governor_proposals['governance'].map(lambda x: x['platform_deployment_id'])
governor_proposals['start_as_epoch'] = governor_proposals['start'].map(lambda x: x['date_as_epoch'])
governor_proposals['end_as_epoch'] = governor_proposals['end'].map(lambda x: x['date_as_epoch'])
# map author out of proposer.author
governor_proposals['author'] = governor_proposals['proposer'].map(lambda x: x['author'])
# map votes_count out of voteTally
governor_proposals['votes_count'] = governor_proposals['voteTally'].map(lambda x: x['votes_count'])
governor_proposals.head()

,governance,proposal_id,start,end,proposer,voteTally,description,platform_deployment_id,start_as_epoch,end_as_epoch,author,votes_count
26,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'date_as_epoch': 1666434539},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124},\n\n# Simple Summary\n\nThis proposal updates ...,e1a6f432-bc3c-438e-a07b-ce35095de79a,1666203011,1666434539,0x55b16934c3661e1990939bc57322554d9b09f262,124
40,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'date_as_epoch': 1678790255},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98},# [S17] Gitcoin Allo Budget Proposal\nThis a r...,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,1678300151,1678790255,0x4be88f63f919324210ea3a2ccad4ff0734425f91,98
58,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'date_as_epoch': 1663550339},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9},IIP-26: Add senior tranche of cpWIN-USDC PYT a...,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,1663340879,1663550339,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,9
82,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'date_as_epoch': 1674702539},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413},\n\n# Simple Summary\n\nThis governance propos...,e1a6f432-bc3c-438e-a07b-ce35095de79a,1674470891,1674702539,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,413
114,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-348,{'date_as_epoch': 1697645219},{'date_as_epoch': 1697877215},{'author': '0x329c54289ff5d6b7b7dae13592c6b1ed...,{'votes_count': 191},"---\ntitle: ""Enable borrow of OP token""\nautho...",e1a6f432-bc3c-438e-a07b-ce35095de79a,1697645219,1697877215,0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4,191


In [16]:
governor_proposals = governor_proposals[['platform_deployment_id', 'proposal_id', 'author', 'start_as_epoch', 'end_as_epoch', 'votes_count', 'description']]
governor_proposals.head()

,platform_deployment_id,proposal_id,author,start_as_epoch,end_as_epoch,votes_count,description
26,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,1666203011,1666434539,124,\n\n# Simple Summary\n\nThis proposal updates ...
40,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,1678300151,1678790255,98,# [S17] Gitcoin Allo Budget Proposal\nThis a r...
58,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,1663340879,1663550339,9,IIP-26: Add senior tranche of cpWIN-USDC PYT a...
82,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,1674470891,1674702539,413,\n\n# Simple Summary\n\nThis governance propos...
114,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-348,0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4,1697645219,1697877215,191,"---\ntitle: ""Enable borrow of OP token""\nautho..."


In [17]:
governor_proposals['platform'] = 'governor'

In [18]:
# parse date from seconds epoch
governor_proposals['start'] = pd.to_datetime(governor_proposals['start_as_epoch'], unit='s')
governor_proposals['end'] = pd.to_datetime(governor_proposals['end_as_epoch'], unit='s')
governor_proposals['date'] = governor_proposals['start']
governor_proposals.head()

,platform_deployment_id,proposal_id,author,start_as_epoch,end_as_epoch,votes_count,description,platform,start,end,date
26,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,1666203011,1666434539,124,\n\n# Simple Summary\n\nThis proposal updates ...,governor,2022-10-19 18:10:11,2022-10-22 10:28:59,2022-10-19 18:10:11
40,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,1678300151,1678790255,98,# [S17] Gitcoin Allo Budget Proposal\nThis a r...,governor,2023-03-08 18:29:11,2023-03-14 10:37:35,2023-03-08 18:29:11
58,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,1663340879,1663550339,9,IIP-26: Add senior tranche of cpWIN-USDC PYT a...,governor,2022-09-16 15:07:59,2022-09-19 01:18:59,2022-09-16 15:07:59
82,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,1674470891,1674702539,413,\n\n# Simple Summary\n\nThis governance propos...,governor,2023-01-23 10:48:11,2023-01-26 03:08:59,2023-01-23 10:48:11
114,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-348,0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4,1697645219,1697877215,191,"---\ntitle: ""Enable borrow of OP token""\nautho...",governor,2023-10-18 16:06:59,2023-10-21 08:33:35,2023-10-18 16:06:59


In [19]:
governor_proposals['title'] = pd.NA
governor_proposals['title'] = governor_proposals['title'].astype(str)

In [20]:
# * platform
# * platform_deployment_id
# * proposal_id
# * author
# * date -- this should be as a pandas date, not as an epoch
# * votes_count

# keep these cols
governor_proposals = governor_proposals[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count', 'title', 'description', 'start', 'end']]
governor_proposals.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end
26,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,2022-10-19 18:10:11,124,<NA>,\n\n# Simple Summary\n\nThis proposal updates ...,2022-10-19 18:10:11,2022-10-22 10:28:59
40,governor,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,2023-03-08 18:29:11,98,<NA>,# [S17] Gitcoin Allo Budget Proposal\nThis a r...,2023-03-08 18:29:11,2023-03-14 10:37:35
58,governor,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,2022-09-16 15:07:59,9,<NA>,IIP-26: Add senior tranche of cpWIN-USDC PYT a...,2022-09-16 15:07:59,2022-09-19 01:18:59
82,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,2023-01-23 10:48:11,413,<NA>,\n\n# Simple Summary\n\nThis governance propos...,2023-01-23 10:48:11,2023-01-26 03:08:59
114,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-348,0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4,2023-10-18 16:06:59,191,<NA>,"---\ntitle: ""Enable borrow of OP token""\nautho...",2023-10-18 16:06:59,2023-10-21 08:33:35


In [21]:
# group by platform_deployment_id and val counts
governor_proposals.groupby('platform_deployment_id')['proposal_id'].count()

platform_deployment_id
1a11c997-d9c2-4d3c-abe4-3d85190d02f2     38
1cb769c9-ac93-4f07-8ae1-26a231a05e84      5
32b7bb94-fbc9-4172-9219-0ed422edf0ce     20
33c90aeb-e7eb-4656-8663-f4831dc63468     17
4e758c1e-8410-4fa4-9924-6bead3afde57     37
4fa685ba-26e9-40cb-b98c-cb9182a3b407     40
535d7de4-0c53-42ab-8e9a-001430a17163     15
56ba6663-b056-435c-846e-09d2ab1cf434     17
5b4e7a42-0f44-4e1b-9297-d006997b2233     13
9611750e-035c-48d1-ac3e-27ba85c1dec9    199
9ed2dea1-f955-4453-b99d-a8bac116bbe4     24
9fc93056-60a4-4972-894f-a3db0733fb37     31
a53c23d6-d21f-408a-906c-1272985c2506     20
ab030329-6200-45a0-aca5-f22aa89581fa     81
b5f50e58-820f-413f-8ba6-84c44c59c1b3      4
bddd5124-ab7f-4911-b961-d923c1dd6085     18
c0b43294-f8b9-4bbb-8a25-b5773d7b23b1     60
c4b7bc0b-887f-4bc6-a165-d4b726e6f35e     22
d897ffd3-8c0d-4a6a-92e0-0c84e0729e61     17
e1a6f432-bc3c-438e-a07b-ce35095de79a    289
e99546c7-58d9-4f78-b3b2-084dfad11436      6
eb93f72a-f8fb-49ff-9a23-4b5f78a4aa04     11
fc98f8f7-

In [22]:
# group by platform_deployment_id and val counts
governor_proposals.groupby('platform_deployment_id')['votes_count'].sum()

platform_deployment_id
1a11c997-d9c2-4d3c-abe4-3d85190d02f2       225
1cb769c9-ac93-4f07-8ae1-26a231a05e84       153
32b7bb94-fbc9-4172-9219-0ed422edf0ce        73
33c90aeb-e7eb-4656-8663-f4831dc63468       117
4e758c1e-8410-4fa4-9924-6bead3afde57       299
4fa685ba-26e9-40cb-b98c-cb9182a3b407     38963
535d7de4-0c53-42ab-8e9a-001430a17163       109
56ba6663-b056-435c-846e-09d2ab1cf434      1999
5b4e7a42-0f44-4e1b-9297-d006997b2233       123
9611750e-035c-48d1-ac3e-27ba85c1dec9     12372
9ed2dea1-f955-4453-b99d-a8bac116bbe4      4634
9fc93056-60a4-4972-894f-a3db0733fb37    500684
a53c23d6-d21f-408a-906c-1272985c2506       308
ab030329-6200-45a0-aca5-f22aa89581fa      1777
b5f50e58-820f-413f-8ba6-84c44c59c1b3        33
bddd5124-ab7f-4911-b961-d923c1dd6085       480
c0b43294-f8b9-4bbb-8a25-b5773d7b23b1      4780
c4b7bc0b-887f-4bc6-a165-d4b726e6f35e        76
d897ffd3-8c0d-4a6a-92e0-0c84e0729e61       188
e1a6f432-bc3c-438e-a07b-ce35095de79a     19127
e99546c7-58d9-4f78-b3b2-084dfad11436 

In [23]:
governor_proposals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 26 to 10735
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   platform                985 non-null    object        
 1   platform_deployment_id  985 non-null    object        
 2   proposal_id             985 non-null    object        
 3   author                  985 non-null    object        
 4   date                    985 non-null    datetime64[ns]
 5   votes_count             985 non-null    int64         
 6   title                   985 non-null    object        
 7   description             985 non-null    object        
 8   start                   985 non-null    datetime64[ns]
 9   end                     985 non-null    datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(6)
memory usage: 84.6+ KB


In [24]:
# save to csv
# governor_proposals.to_csv('governor_proposals.csv', index=False)
governor_proposals.to_parquet('governor_proposals.parquet', index=False)